In [3]:
import gradio as gr
import torch
from models import vgg19
from PIL import Image
from torchvision import transforms
import gradio as gr
import cv2
import numpy as np
#create a folder named pretrained_models
#download the model using the url and save it in the above folder
url = "https://drive.google.com/file/d/11wsCnalTaElaAcXQFVMq4T7BKW6PLxnC/view?usp=sharing"
model_path = "pretrained_models/model_sh_B.pth"
#model_path = r"D:\2023\deploy cc model using gradio\original dm count\DM-Count-master\pretrained_models\model_sh_B.pth"
device = torch.device('cpu')  # device can be "cpu" or "gpu"

model = vgg19()
model.to(device)
model.load_state_dict(torch.load(model_path, device))
model.eval()


def predict(inp):
    inp = Image.fromarray(inp.astype('uint8'), 'RGB')
    #inp = transforms.ToTensor()(inp).unsqueeze(0)
    trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    # Apply the transformation to the image
    img_tensor = trans(inp)
    # extra dimension for batch size
    inp = img_tensor.unsqueeze(0)
    inp = inp.to(device)

    with torch.set_grad_enabled(False):
        outputs, _ = model(inp)

    count = torch.sum(outputs).item()
    vis_img = outputs[0, 0].cpu().numpy()
    # normalize density map values from 0 to 1, then map it to 0-255.
    vis_img = (vis_img - vis_img.min()) / (vis_img.max() - vis_img.min() + 1e-5)
    vis_img = (vis_img * 255).astype(np.uint8)
    vis_img = cv2.applyColorMap(vis_img, cv2.COLORMAP_JET)
    vis_img = cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB)
    return vis_img, int(count)


title = "Crowd Counter"
desc = "A demo of Crowd counting model"
examples = [
    ["images\IMG_1.jpg"],
    ["images\IMG_2.jpg"],
    ["images\IMG_107.jpg"],
]
inputs = gr.inputs.Image(label="Image of Crowd")
outputs = [gr.outputs.Image(label="Predicted Density Map",type = "numpy"), gr.outputs.Label(label="Predicted Count",type = "numpy")]
gr.Interface(fn=predict, inputs=inputs, outputs=outputs, title=title, description=desc, examples=examples,
             allow_flagging=False).launch()

C:\Users\Cyntexia\AppData\Local\Temp\ipykernel_15040\708864498.py:49: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs = gr.inputs.Image(label="Image of Crowd")
C:\Users\Cyntexia\AppData\Local\Temp\ipykernel_15040\708864498.py:49: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs = gr.inputs.Image(label="Image of Crowd")
C:\Users\Cyntexia\AppData\Local\Temp\ipykernel_15040\708864498.py:50: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs = [gr.outputs.Image(label="Predicted Density Map",type = "numpy"), gr.outputs.Label(label="Predicted Count",type = "numpy")]
C:\Users\Cyntexia\AppData\Local\Temp\ipykernel_15040\708864498.py:50: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove t

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


c:\Users\Cyntexia\anaconda3\envs\gradio_app\Lib\site-packages\torch\nn\functional.py:4079: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
c:\Users\Cyntexia\anaconda3\envs\gradio_app\Lib\site-packages\torch\nn\functional.py:4079: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
